## Importing Libraries

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import py21cmfast as p21c
from datetime import datetime
import logging, os
from numba import njit, jit

## Set logger to log caching activity

In [2]:
logger = logging.getLogger('21cmFAST')
logger.setLevel(logging.INFO)

## Version of 21cmFAST

In [3]:
print(f"Using 21cmFAST version {p21c.__version__}")

Using 21cmFAST version 3.0.0.dev5


## Number of cores running

In [4]:
print(f'Number of threads running = {os.cpu_count()}')

Number of threads running = 16


## Reset cache location 

In [5]:
p21c.config['direc'] = '/lustre/aoc/projects/hera/wchin/21cmFAST-cache'

## Cosmological Parameters (Default is used when no input is specified)

In [6]:
cosmo_params = p21c.CosmoParams()

## User Parameters, like box length, number of voxels (i.e. resolution) etc.

In [7]:
BOX_LEN=301  # 300, 301
HII_DIM=301  # 450, 301

user_params = p21c.UserParams(
    BOX_LEN=BOX_LEN,  # Box length in Mpc
    DIM=4*HII_DIM,      # Number of Voxels for hight resolution 
    HII_DIM=HII_DIM,  # Number of Voxels for low resolution 
    N_THREADS=os.cpu_count()
)

## Creating initial conditions box

In [8]:
start_time = datetime.now()
print(f'Excution qued at {start_time}')

init_cond = p21c.initial_conditions(
    cosmo_params=cosmo_params,
    user_params=user_params,
    direc='/lustre/aoc/projects/hera/wchin/21cmFAST-cache'
)

end_time = datetime.now()
execution_time = end_time - start_time
print(f'Execution completed at {end_time}')
print(f'Execution time = {execution_time}')

Excution qued at 2020-12-26 19:57:31.095330


2020-12-26 20:00:12,567 | INFO | Existing init_boxes found and read in (seed=230806296593).


Execution completed at 2020-12-26 20:00:12.569082
Execution time = 0:02:41.473752


## Make sure averaging region size is not larger than the box itself 

In [9]:
def check_averaging_radius_limit(averaging_radius, box_length):
    # check to see if averaging region is larger than the box itself
    if averaging_radius > (box_length-1)/2:
        raise ValueError(f'Averaging_radius = {averaging_radius} > \
{(box_length-1)/2} = (Box Length-1)/2, averaging region is larger than the box itself.')

## Measure the distance of each voxel to the center

In [10]:
@jit
def distance_from_coordinate(box_length):
    """
    Generate a cube of voxels.
    On each voxel, the distanace from the center is 
    calculated and the value is assigned to the voxel.
    jit by numba compiles what it can to machine code,
    the rest as python code.
    
    Parameters
    ----------
    box_length : int
        The length of each side of the cube.
        
    Returns
    -------
    distance : 3D-ndarray
        Cube of voxels with each voxel having its 
        distance from the center assigned to it.
    """
    # range of nummbers with 0 as the center    
    index = np.arange(-0.5*(box_length-1), 0.5*(box_length+1))
    # 3D mesh
    x_mesh, y_mesh, z_mesh = np.meshgrid(index, index, index, indexing='ij')
    # generating cube and computing distacne for each voxel
    distance = np.sqrt((x_mesh)**2 + (y_mesh)**2 + (z_mesh)**2)
    
    return distance

## Random Coordinate

In [11]:
def random_cube_regions(box_length, number_of_coordinates, radius):
    """
    Selects a number cubical sub-regions
    within a larger cube at random positions.
    
    Parameters
    ----------
    box_length            : int
        Side length of the larger cube in
        arbitrary units of number of cells.
    number_of_coordinates : int
        Νumber of sub-regions to be selected.
    radius                : int
        Side of the smaller selected cubical 
        sub-region in arbitrary units of number of cells.
        Called radius because a sphere is
        often define within this smaller cube.
        
    Returns
    -------
    inds1 : ndarray dtype:int
        In 1D, the left bound that defines the cubical region.
    inds2 : ndarray dtype:int
        In 1D, the right bound that defines the cubical region.
    """
    
    np.random.seed()  # no entry: set seed to a randome number
#     np.random.seed(4)  # specifying seed for testing purposes

    # selecting 'number_of_coordiantes' random coordinates within larger cube
    coordinates = np.random.randint(0, box_length, size=(number_of_coordinates, 3))
    
    # cube indices 
    inds1 = (coordinates-radius).astype(int)
    inds2 = (coordinates+radius+1).astype(int)  # ending index is not inclusive
    

    return inds1, inds2

## Select a Smaller Cube with Sides 2R+1 Voxels, Centered about the Random Coordinate

In [12]:
def slicing_the_cube(ind1, ind2, box):
    """
    Selects a smaller cubical sub-region within a larger cube called 'box'.
    Incoorporates periodic boundary conditions, i.e. Pac-Man effect.
    This function takes in a set of slicing indices of one particular randomly
    selected cubical sub-region and returns the selected smaller cube.
    
    Parameters
    ----------
    ind1 : 1D ndarray, dtype: int
        The left bounds of the selected region in 1D respectively.
    ind2 : 1D ndarray, dtype: int
        The right bounds of the selected region in 1D respectively.
    box  : 3D ndarray, dtype: float32
        Data cube, when the smaller cubical
        sub-regions are being selected from.
        
    Returns
    -------
    output_box : 3D ndarray, dtype: float32
        The selected cubical smaller sub-region
        within the larger data cube.
    """
        
    if ind1[0] < 0:  # periodic boundary conditions
        # region that went beyond the zeroth voxel face of the
        # cube is replaced by the region at the 'box_length'th
        # voxel face of the cube with the same size.
        x_inds = np.r_[(ind1[0]+len(box)):len(box), 0:ind2[0]]
    elif ind2[0] > len(box):
        # region that went beyond the 'box_length'th voxel face
        # of the cube is replaced by the region at the zeroth 
        # voxel face of the cube with the same size.
        x_inds = np.r_[ind1[0]:len(box), 0:(ind2[0]-len(box))]
    else:
        # selected voxel is perfectly in the larger data cube.
        x_inds = np.r_[ind1[0]:ind2[0]]

    if ind1[1] < 0:
        y_inds = np.r_[(ind1[1]+len(box)):len(box), 0:ind2[1]]
    elif ind2[1] > len(box):
        y_inds = np.r_[ind1[1]:len(box), 0:(ind2[1]-len(box))]
    else:
        y_inds = np.r_[ind1[1]:ind2[1]]

    if ind1[2] < 0:
        z_inds = np.r_[(ind1[2]+len(box)):len(box), 0:ind2[2]]
    elif ind2[2] > len(box):
        z_inds = np.r_[ind1[2]:len(box), 0:(ind2[2]-len(box))]
    else:
        z_inds = np.r_[ind1[2]:ind2[2]]
                    
    try:
        # box[indices]
        output_box = box[np.ix_(x_inds, y_inds, z_inds)]
        
    except IndexError:  # sample region larger than box.
        print(f'ind1 = {ind1}')  # print useful info
        print(f'ind2 = {ind2}')  # for debugging
        print(f'box length = {len(box)}')
        print(f'x_ind1 = {ind1[0]}')
        print(f'x_ind2 = {ind2[0]}')
        print(f'x_inds = {x_inds}')
        print(f'y_ind1 = {ind1[1]}')
        print(f'y_ind2 = {ind2[1]}')
        print(f'y_inds = {y_inds}')
        print(f'z_ind1 = {ind1[2]}')
        print(f'z_ind2 = {ind2[2]}')
        print(f'z_inds = {z_inds}')
        
    return output_box

## Gausssian Sphere Averaging

In [13]:
def gaussian_sphere_average(
    distance_box, 
    radius, 
    input_box, 
    shell_num, 
    sigma_factor
):
    """
    Takes in a cube of voxels and defines 'shell_num'
    of concentric spherical shells and a core.
    A mean is taken over the voxels in each shell and the core.
    Then 'shell_num' equally spaced values are drawn as weights from 0 to 
    'sigma_factor' standard deviations of the Gaussian distribution.
    Each spherical shell and the core is assigned a weight, 
    The inner most core is weighted the most.
    The weights decrease moving outwards through the shells.
    And the outmost shell is weighted the least.
    The function then computes a weighted average over the shells and core.
    
    Parameters
    ----------
    distance_box : 3D ndarray, dtype: float32
        Cube of voxels with each voxel having its 
        distance from the center assigned to it.
        This function uses the distance box to
        set the condition required to slice
        the voxels in the spherical shells.
    radius : int
        Radius of the Guassian sphere, i.e. 
        the sphere complied by the center core 
        and the concentric spherical shells 
        in units of number of voxels.
    input_box : 
    
    
    
    """
    
    
    mean = np.zeros(shell_num)

    shell_radius_edges = np.linspace(0,1,shell_num+1)
    # sigma_factor number of sigmas the weighting goes out to, sigma = radius
    shell_center = 0.5*(shell_radius_edges[1:] + shell_radius_edges[:-1])*sigma_factor 
    weight = gaussian(x=shell_center)
    
    for ii in range(shell_num):
        condition = np.logical_and(
            distance_box <= shell_radius_edges[ii+1]*radius, 
            distance_box > shell_radius_edges[ii]*radius
        )
        mean[ii] = np.mean(input_box[condition])  # inside shell mean
        
    Gaussian_mean = np.average(mean, weights=weight)
    
    return Gaussian_mean

## Gaussian function

In [14]:
@njit
def gaussian(x):  # μ=0, σ=1/sqrt(2), π=1
    """
    Gaussian distribution with amplitude=1,
    mu=0, standard deviation=1/sqrt(2), pi=1.
    @njit by numba compiles the code in machine code 
    instead of python, speeding up the computation.
    
    Parameters
    ----------
    x        : array_like
        Independent variable
        
    Returns
    -------
    gaussian : array_like
        Gaussian distribution array
    """
    gaussian = np.exp(-x**2)
    return gaussian

## Sphere Blurring Function

In [15]:
@jit
def average_neutral_fraction_distribution(
    box, 
    radius, 
    iteration, 
    shell_num=6, 
    sigma_factor=1.4370397097748921*3, 
    blur_shape=None
):
    
    box = box.copy()  # make copy of input box to have a separate box
    
    mean_data = np.zeros(iteration)  # empty list for data collection
    
    
    if blur_shape == 'Gaussian_sphere':
        
        
# ====================================================================================================================
        # Radius Ratio 1
        radius = int(round(radius*1.4370397097748921*3))
        
        # Radius Ratio 2
#         radius = int(round(radius*((4/3/np.sqrt(np.pi))**(1/3))*13/4))  
            # 13/4 --> speculated correction factor
# ====================================================================================================================

        
        
        # check to see if averaging region is largert than the box itself
        check_averaging_radius_limit(radius, len(box))
            
        # used as condition to define a sphere within a cube
        dist_frm_coord_box = distance_from_coordinate(radius*2+1)
        
        # iteration number of random cube region indices in the box
        rand_coord_inds1, rand_coord_inds2 = random_cube_regions(
            len(box), 
            iteration, 
            radius
        )  

        
        for i in range(iteration):
            cube_region_box = slicing_the_cube(
                rand_coord_inds1[i, :], 
                rand_coord_inds2[i, :], 
                box
            )
            # mean
            mean_data[i] = gaussian_sphere_average(
                dist_frm_coord_box, 
                radius, 
                cube_region_box, 
                shell_num, 
                sigma_factor
            )
        
    elif blur_shape == 'top_hat_sphere':
        
        # check to see if averaging region is largert than the box itself
        check_averaging_radius_limit(radius, len(box))
            
        # used as condition to define a sphere within a cube
        dist_frm_coord_box = distance_from_coordinate(radius*2+1)
        
        # iteration number of random cube region indices in the box
        rand_coord_inds1, rand_coord_inds2 = random_cube_regions(
            len(box), 
            iteration, 
            radius
        )  
        
        for i in range(iteration):
            cube_region_box = slicing_the_cube(
                rand_coord_inds1[i, :], 
                rand_coord_inds2[i, :], 
                box
            )
            # mean
            mean_data[i] = top_hat_sphere_average(
                dist_frm_coord_box, 
                radius, 
                cube_region_box
            )
            
    elif blur_shape == 'top_hat_cube':
                
        # ratio determiend by equating the volumes of cube & sphere
        radius = int(round((radius*((4*np.pi/3)**(1/3))-1)/2))  

        # check to see if averaging region is largert than the box itself
        check_averaging_radius_limit(radius, len(box))

        # iteration number of random cube region indices in the box
        rand_coord_inds1, rand_coord_inds2 = random_cube_regions(
            len(box), 
            iteration, 
            radius
        )  
        
        for i in range(iteration):
            cube_region_box = slicing_the_cube(
                rand_coord_inds1[i, :], 
                rand_coord_inds2[i, :], 
                box
            )
            # mean
            mean_data[i] = top_hat_cube_average(cube_region_box)
    else:
        
        print('Blurring shape assumed to be a Gaussian sphere with 4 shells \
              weighted by equally spaced values from 0 sigma to 4 sigma.')
                
        
# ====================================================================================================================
        # Radius Ratio 1
        radius = int(round(radius*1.4370397097748921*3))
        
        # Radius Ratio 2
#         radius = int(round(radius*((4/3/np.sqrt(np.pi))**(1/3))*13/4))  
            # 13/4 --> speculated correction factor
# ====================================================================================================================
        
        
        # check to see if averaging region is largert than the box itself
        check_averaging_radius_limit(radius, len(box))

        # used as condition to define a sphere within a cube
        dist_frm_coord_box = distance_from_coordinate(radius*2+1)

        # iteration number of random cube region indices in the box
        rand_coord_inds1, rand_coord_inds2 = random_cube_regions(
            len(box), 
            iteration, 
            radius
        )  
        
        for i in range(iteration):
            cube_region_box = slicing_the_cube(
                rand_coord_inds1[i, :], 
                rand_coord_inds2[i, :], 
                box
            )
            mean_data[i] = gaussian_sphere_average(
                dist_frm_coord_box, 
                radius, 
                cube_region_box, 
                shell_num, 
                sigma_factor
            )
            # mean
            
    return mean_data

## Generate Average Neutral Fraction Distributions as a function of redshift

In [16]:
def generate_distributions(
    boxes,
    radii=np.arange(8, 17, 1),
    iterations=int(10**3),
    sigma_factor=1.4370397097748921*3,
    shell_number=6,
    progress_status=True
):

    gaussians = np.zeros((len(boxes), len(radii), iterations))
    
    # print progress and local time
    if progress_status:
        start_time = datetime.now()
        current_time = start_time
        print(f'Progress = 0%, localtime = {start_time}')

    for i, box in enumerate(boxes):
               
        for ii, radius in enumerate(radii):
                        
            gaussians[i, ii, :] = average_neutral_fraction_distribution(
                box=box,
                radius=radius,
                iteration=iterations,
                sigma_factor=sigma_factor,
                shell_num=shell_number,
                blur_shape='Gaussian_sphere'
            )
        # print progress and local time
        if progress_status:
            previous_time = current_time
            current_time = datetime.now()
            loop_time = current_time - previous_time
            elapsed_time = current_time - start_time
            print(f'progress = {int(round((i+1)*100/len(boxes)))}%, \
localtime = {current_time}, loopexecuted in {loop_time}, elapsedtime = {elapsed_time}')
    
    return gaussians

## Vary: Rmax, EFF, constant: x_HI, z. x_HI error: 1e-2%

In [17]:
R_BUBBLE_MAXES = np.linspace(30, 0.225, 9)
HII_EFF_FACTORS = np.array(
    [19.04625, 
     19.511249999999997, 
     20.23875, 
     21.085, 
     22.655000000000012, 
     25.779375, 
     32.056640625, 
     56.6734375, 
     5291.5]
)
redshifts = np.array([6]*len(R_BUBBLE_MAXES))

## Generate ionized boxes and total neutral fractions as a function of redshift

In [18]:
# R_BUBBLE_MAXES = np.array(
#     [10, 20, 30]*3
# )
# HII_EFF_FACTORS = np.array(
#     [62.625, 42.042500000000004, 36.183125000000004,
#      47.1875, 34.390625, 31.25, 
#      37.96875, 29.53125, 27.6875]  
# )
# redshifts = np.array([7]*len(R_BUBBLE_MAXES))

progress_status = True



# sample_range = 10
# target_variable = 0.3  # x_HI neutral fraction
# target_error = 0.5  # percent
# error = 1  # intializing
# while abs(error) > target_variable*target_error/100:



ionized_boxes = np.zeros((len(redshifts), HII_DIM, HII_DIM, HII_DIM))
total_neutral_fractions = np.zeros(len(redshifts))

# print progress and local time
if progress_status:
    start_time = datetime.now()
    current_time = start_time
    print(f'Progress = 0%, localtime = {start_time}')

for i, z in enumerate(redshifts):
    ionized_boxes[i] = p21c.ionize_box(
        redshift=z, 
        init_boxes=init_cond,
        astro_params={
            'HII_EFF_FACTOR': HII_EFF_FACTORS[i],
            'R_BUBBLE_MAX': R_BUBBLE_MAXES[i]
        }
    ).xH_box
    total_neutral_fractions[i] = np.mean(ionized_boxes[i])

    # print progress and local time
    if progress_status:
        previous_time = current_time
        current_time = datetime.now()
        loop_time = current_time - previous_time
        elapsed_time = current_time - start_time
        print(f'progress = {int(round((i+1)*100/len(redshifts)))}%, \
localtime = {current_time}, loopexecuted in {loop_time}, elapsedtime = {elapsed_time}')
        
total_neutral_fractions



#     previous_error=error
#     error = target_variable - total_neutral_fractions[0]
    
#     print(f'HII_EFF_FACTOR={HII_EFF_FACTORS[0]}')
#     print(f'x_HI={total_neutral_fractions[0]}')
#     print(f'sample_range={sample_range}')
#     print(f'previous_error={previous_error}')
#     print(f'error={error}')
    
#     if error > 0:
#         HII_EFF_FACTORS[0] -= sample_range
#     else:
#         HII_EFF_FACTORS[0] += sample_range
        
#     if error*previous_error < 0:
#         sample_range -= 0.5*sample_range

Progress = 0%, localtime = 2020-12-26 20:00:12.732264


2020-12-26 20:00:14,738 | INFO | Existing z=6 ionized boxes found and read in (seed=230806296593).


progress = 11%, localtime = 2020-12-26 20:00:14.819977, loopexecuted in 0:00:02.087713, elapsedtime = 0:00:02.087713


2020-12-26 20:00:16,685 | INFO | Existing z=6 ionized boxes found and read in (seed=230806296593).


progress = 22%, localtime = 2020-12-26 20:00:16.763119, loopexecuted in 0:00:01.943142, elapsedtime = 0:00:04.030855


...

2020-12-26 20:00:27,863 | INFO | Existing z=6 ionized boxes found and read in (seed=230806296593).


progress = 89%, localtime = 2020-12-26 20:00:27.948261, loopexecuted in 0:00:01.779584, elapsedtime = 0:00:15.215997


2020-12-26 20:00:31,212 | INFO | Existing z=6 ionized boxes found and read in (seed=230806296593).


progress = 100%, localtime = 2020-12-26 20:00:31.535352, loopexecuted in 0:00:03.587091, elapsedtime = 0:00:18.803088


array([0.19999881, 0.19998097, 0.20000417, 0.20001106, 0.19998624,
       0.20001978, 0.19999591, 0.19998911, 0.19998213])

## Generate the distributions and storing the data in global variables

In [19]:
radii = np.array([13]*2)  # array([34, 32, 30,... 6, 4]) units: voxels (51, 3, -3)
iterations = int(1e5)

gaussians = generate_distributions(
    boxes=ionized_boxes[[-3, -2]],
    radii=radii,
    iterations=iterations,
    sigma_factor=1.4370397097748921*3
)

Progress = 0%, localtime = 2020-12-26 20:00:31.910243


<ipython-input-15-521f3fee503c>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "average_neutral_fraction_distribution" failed type inference due to: Untyped global name 'check_averaging_radius_limit': cannot determine Numba type of <class 'function'>

File "<ipython-input-15-521f3fee503c>", line 31:
def average_neutral_fraction_distribution(
    <source elided>
        # check to see if averaging region is largert than the box itself
        check_averaging_radius_limit(radius, len(box))
        ^

  @jit
<ipython-input-15-521f3fee503c>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "average_neutral_fraction_distribution" failed type inference due to: cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-15-521f3fee503c>", line 21:
def average_neutral_fraction_distribution(
    <source elided>
        # Radius Ratio 1
        radiu

progress = 50%, localtime = 2020-12-26 21:14:57.687034, loopexecuted in 1:14:25.776791, elapsedtime = 1:14:25.776791
progress = 100%, localtime = 2020-12-26 22:27:28.591251, loopexecuted in 1:12:30.904217, elapsedtime = 2:26:56.681008


In [20]:
np.save('/lustre/aoc/projects/hera/wchin/gaussians_Rmax7.67,3.95_r13', gaussians)

In [22]:
radii = np.array([13, 12, 11, 10])  # array([34, 32, 30,... 6, 4]) units: voxels (51, 3, -3)
iterations = int(1e5)

gaussians = generate_distributions(
    boxes=np.array([ionized_boxes[0]]),
    radii=radii,
    iterations=iterations,
    sigma_factor=1.4370397097748921*3
)

Progress = 0%, localtime = 2020-12-23 14:26:17.870783
progress = 100%, localtime = 2020-12-23 16:36:44.880486, loopexecuted in 2:10:27.009703, elapsedtime = 2:10:27.009703


np.save('/lustre/aoc/projects/hera/wchin/gaussians_Rmax30_r13,12,11,10', gaussians)

gaussians = np.load('/lustre/aoc/projects/hera/wchin/VaryRmax&EFF_gaussians.npy')

## Check if object is iterable

In [ ]:
def iterable(obj):
    
    try:
        iter(obj)
        
    except Exception:
        return False
    
    else:
        return True

## Histogram Function

In [ ]:
def histogram(
    y1s, 
    figure_shape, 
    y2s=None,
    y3s=None,
    marker_lines=None,
    y1s_labels=None,
    y2s_label=None,
    y3s_label=None,
    title=None, 
    legend_font_size=12,
    fancy_legend_box=True,
    legend_alpha=0.5,
    shared_title=None,
    shared_title_x_position=0.5,   # figure coordinates, max=1 I think
    shared_title_y_position=0.92,
    shared_x_label=None, 
    shared_x_label_x_position=0.5,
    shared_x_label_y_position=0.08,
    shared_y_label=None, 
    shared_y_label_x_position=0.07,
    shared_y_label_y_postion=0.5,
    x_start=0, 
    x_stop=1, 
    bin_num=int(1e3), 
    color='white', 
    figure_size=(18,7), 
    font_size=15, 
    horizontal_gap=0.05, 
    vertical_gap=0.05, 
    y_scale='linear', 
    y_notation='plain', 
    share_x_axis=True, 
    share_y_axis=True,
    dpi=100
):  # a: x start, b: x stop
    
    bin_edges = np.linspace(x_start, x_stop, bin_num) # bin_num of bins from 0-1
    
    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    
    if np.array(y1s).ndim > 2:  # hopefully more robust condition.
        
        fig, axes = plt.subplots(
            figure_shape[0], 
            figure_shape[1], 
            figsize=figure_size, 
            sharex=share_x_axis, 
            sharey=share_y_axis, 
            gridspec_kw={"hspace":vertical_gap, 'wspace':horizontal_gap},
            dpi=dpi
        )
        
        if shared_title != None:
            
            fig.suptitle(
                x=shared_title_x_position, 
                y=shared_title_y_position, 
                t=shared_title, ha='center', 
                size=1.5*font_size, 
                color=color
            )
            
        if shared_x_label != None:  # shared x label
            fig.text(
                x=shared_x_label_x_position, 
                y=shared_x_label_y_position, 
                s=shared_x_label, ha='center', 
                size=font_size, 
                color=color
            )

        if shared_y_label != None:  # shared y label
            fig.text(
                x=shared_y_label_x_position, 
                y=shared_y_label_y_postion, 
                s=shared_y_label, 
                va='center', 
                rotation='vertical', 
                size=font_size, 
                color=color
            )
        
        for i, y1 in enumerate(y1s):
            
            for ii, marker_line in enumerate(marker_lines):
                axes.flatten()[i].plot(
                    bin_centers, 
                    np.histogram(y1s[i,ii,:], bins=bin_edges)[0], 
                    marker_line,
                    label=y1s_labels[ii]
                )
            
            if iterable(y2s):
                axes.flatten()[i].plot(
                    bin_centers, 
                    np.histogram(y2s[i], bins=bin_edges)[0], 
                    '--', 
                    label=y2s_label
                )
                
                if iterable(y3s):
                    axes.flatten()[i].plot(
                        bin_centers, 
                        np.histogram(y3s[i], bins=bin_edges)[0], 
                        label=y3s_label
                    )
                    
                    
            if y1s_labels != None:
                axes.flatten()[i].legend(
                    prop={'size': legend_font_size}, 
                    fancybox=fancy_legend_box, 
                    framealpha=legend_alpha
                )
                
            if title != None:
                axes.flatten()[i].set_title(title[i], color=color, fontsize=font_size)
                
            axes.flatten()[i].set_yscale(y_scale)
            
            if y_notation == 'sci':
                axes.flatten()[i].ticklabel_format(
                    axis='y', 
                    style=y_notation, 
                    scilimits=(0,0), 
                    useMathText=True
                )
                
            axes.flatten()[i].tick_params(
                color=color, 
                labelcolor=color, 
                labelsize=font_size, 
                size=font_size
            )  # font style

            for spine in axes.flatten()[i].spines.values():  # figure color
                spine.set_edgecolor(color)
                
    else:
        
        fig, ax = plt.subplots(figsize=figure_size)
        
        ax.plot(bin_centers, np.histogram(y1s, bins=bin_edges)[0], label=y1s_labels)
        
        if y2s != None:
            ax.plot(bin_centers, np.histogram(y2s, bins=bin_edges)[0], '--', label=y2s_label)
            
            if y3s == None:
                ax.legend(
                    prop={'size': legend_font_size}, 
                    fancybox=fancy_legend_box, 
                    framealpha=legend_alpha
                )                 
                
            else:
                ax.plot(bin_centers, np.histogram(y3s, bins=bin_edges)[0], '-.', label=y3s_label)
                ax.legend(
                    prop={'size': legend_font_size}, 
                    fancybox=fancy_legend_box, 
                    framealpha=legend_alpha
                )                 
                
        if title != None:
            ax.set_title(title, color=color, fontsize=font_size)
            
        ax.set_yscale(y_scale)
        
        if y_notation == 'sci':
            ax.ticklabel_format(axis='y', style=y_notation, scilimits=(0,0), useMathText=True)
            
        ax.tick_params(color=color, labelcolor=color, labelsize=font_size)  # font style

        for spine in ax.spines.values():  # figure color
            spine.set_edgecolor(color)
            
    plt.show()

## Creating Histogram

In [ ]:
bins = int(1e2)
histogram(
    y1s=gaussians,       
    marker_lines=['-']*len(radii),
    y1s_labels=[f'Radius={r*BOX_LEN/HII_DIM:.0f}Mpc' for r in radii],
    legend_alpha=0,
    bin_num=bins,   
    title=[f'Rmax={R_BUBBLE_MAXES[i]:.2f}, EFF={HII_EFF_FACTORS[i]:.2f}' for i,
           z in enumerate(redshifts)],
    shared_title=f'Distribution of Average Neutral Fraction \
({bins} bins, {iterations:.2e} iterations)',
    shared_y_label='Counts',
    shared_x_label='Neutral Fraction',
    figure_shape=(3,3), 
    figure_size=(18,18),
    vertical_gap=0.1,
    horizontal_gap=0.05,
    y_scale='log',
#     y_notation='sci',
    share_y_axis=True,
#     dpi=1000
)

In [ ]:
np.shape(gaussians)

In [ ]:
radius = 2
color = 'w'

plt.figure(dpi=300)
bin_edges = np.linspace(0., 1., int(1e2)) # bin_num of bins from 0-1
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

for i in range(9):
    plt.semilogy(
        bin_centers, 
        np.histogram(gaussians[i, radius], bins=bin_edges)[0],
        label=f'Rmax={R_BUBBLE_MAXES[i]:.1f}, EFF={HII_EFF_FACTORS[i]:.1f}'
    )
plt.legend(fancybox=True, framealpha=0.5)
plt.title(f' Radius = {radii[radius]} Mpc', color=color)
plt.tick_params(color=color, labelcolor=color)
plt.show()

In [ ]:
radius = 2
color = 'w'

plt.figure(dpi=300)
bin_edges = np.linspace(0., 1., int(1e2)) # bin_num of bins from 0-1
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

for i in range(5, 9):
    plt.semilogy(
        bin_centers, 
        np.histogram(gaussians[i, radius], bins=bin_edges)[0],
        label=f'Rmax={R_BUBBLE_MAXES[i]:.1f}, EFF={HII_EFF_FACTORS[i]:.1f}'
    )
plt.legend(fancybox=True, framealpha=0.5)
plt.title(f'Averaging Sphere Radius = {radii[radius]} Mpc', color=color)
plt.tick_params(color=color, labelcolor=color)
plt.show()